In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

## Sample preparation

In [ ]:
import generator

In [ ]:
shape = (500, 500)
porsty = 0.3
blobns = 2
noise_prob = 0.05
num_of_angles = 180
tag = 'train'

In [ ]:
orig_phantom_train, proc_phantom_train = generator.create_phantom_and_process(shape, porsty, blobns, noise_prob, num_of_angles, tag)

In [ ]:
tag = 'test'
orig_phantom_test, proc_phantom_test = generator.create_phantom_and_process(shape, porsty, blobns, noise_prob, num_of_angles, tag)

## View results

In [ ]:
import data_manager as dm
dm.show_data_info()

In [ ]:
import pixel_counter as pc

tag = 'train'
indx = 1
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)

## TRAIN

In [ ]:
wtr = 'csv'
pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_train = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_train[0:5]

In [ ]:
Y_train = np.asarray(pixels_df['pixel_real_value'])
Y_train[0:5]

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.StandardScaler().fit(X_train).transform(X_train)
X_train[0:5]

## TEST

In [ ]:
tag = 'test'
indx = 2
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)
wtr = 'csv'

pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_test = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
Y_test = np.asarray(pixels_df['pixel_real_value'])
Y_test[0:5]

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test[0:5]

<h2 id="modeling">Modeling (Logistic Regression with Scikit-learn)</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train, Y_train)

Prediction:

In [ ]:
yhat = LR.predict(X_test)
print(len(yhat))
print(len(X_test))

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(Y_test, yhat, average='weighted')

## Making 2d image

In [ ]:
a = np.asarray([1,2,3,4,5,6,7,8,9])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def to_img(lin_arr, r = False):
    _, ax = plt.subplots(figsize = (10,10))
    size = int(round(np.sqrt(len(lin_arr))))
    img = lin_arr.reshape((size, size))
    ax.imshow(img, cmap = 'gray')
    if r:
        return img

Original img

In [ ]:
to_img(Y_test)

Binarized img

In [ ]:
to_img(yhat)

In [ ]:
jaccard_score(Y_test, yhat, average='weighted')

## Gray img

In [ ]:
X_test = np.asarray(pixels_df[['proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
img = to_img(X_test, True)

In [ ]:
def binarize(signal, threshold=0.5):
    signal = np.asarray(signal)
    return signal > threshold

In [ ]:
bin_img = binarize(img)
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bin_img, cmap = 'gray')

In [ ]:
Y_binarized = np.concatenate(bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized, average='weighted')

## Otsu

In [ ]:
def otsu(gray):
    pixel_number = gray.shape[0] * gray.shape[1]
    mean_weigth = 1.0/pixel_number
    his, bins = np.histogram(gray, np.arange(0,257))
    final_thresh = -1
    final_value = -1
    intensity_arr = np.arange(256)
    for t in bins[1:-1]: # This goes from 1 to 254 uint8 range (Pretty sure wont be those values)
        pcb = np.sum(his[:t])
        pcf = np.sum(his[t:])
        Wb = pcb * mean_weigth
        Wf = pcf * mean_weigth

        mub = np.sum(intensity_arr[:t]*his[:t]) / float(pcb)
        muf = np.sum(intensity_arr[t:]*his[t:]) / float(pcf)
        #print mub, muf
        value = Wb * Wf * (mub - muf) ** 2

        if value > final_value:
            final_thresh = t
            final_value = value
    final_img = gray.copy()
    print(final_thresh)
    final_img[gray > final_thresh] = 255
    final_img[gray < final_thresh] = 0
    return final_img

In [ ]:
otsu_bin_img = otsu(img)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(otsu_bin_img, cmap = 'gray')

In [ ]:
Y_binarized_otsu = np.concatenate(otsu_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_otsu, average='weighted')

## Gaus+binarization

In [ ]:
from scipy.ndimage import gaussian_filter as gfilter

In [ ]:
gaus_bin_img = binarize(gfilter(img, 3))

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(gaus_bin_img, cmap = 'gray')

original:

In [ ]:
to_img(Y_test)

In [ ]:
Y_binarized_gaus = np.concatenate(gaus_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_gaus, average='weighted')

In [ ]:
import cv2
bilateral = cv2.bilateralFilter(np.asarray(img, dtype = np.uint8), 7, 1, 1)
Y_bilaterial = np.concatenate(bilateral)
jaccard_similarity_score(Y_test, Y_bilaterial)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bilateral, cmap = 'gray')

----------

## ADD MORE DATA AND REPEAT

In [ ]:
shape = (500, 500)
porsty = 0.2
blobns = 4
noise_prob = 0
num_of_angles = 360
tag = 'train'

In [ ]:
orig_phantom_train, proc_phantom_train = generator.create_phantom_and_process(shape, porsty, blobns, noise_prob, num_of_angles, tag)

In [ ]:
tag = 'test1'
orig_phantom_test, proc_phantom_test = generator.create_phantom_and_process(shape, porsty, blobns, noise_prob, num_of_angles, tag)

In [ ]:
tag = 'test2'
orig_phantom_test, proc_phantom_test = generator.create_phantom_and_process(shape, porsty, blobns, noise_prob, num_of_angles, tag)

## View results

In [ ]:
import data_manager as dm
dm.show_data_info()

In [ ]:
tag = 'train'
indx = 1
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)

## TRAIN

In [ ]:
wtr = 'csv'
pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_train = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_train[0:5]

In [ ]:
Y_train = np.asarray(pixels_df['pixel_real_value'])
Y_train[0:5]

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.StandardScaler().fit(X_train).transform(X_train)
X_train[0:5]

## TEST 1

In [ ]:
tag = 'test1'
indx = 4
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)
wtr = 'csv'

pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_test = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
Y_test = np.asarray(pixels_df['pixel_real_value'])
Y_test[0:5]

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test[0:5]

## TEST 2

In [ ]:
tag = 'test2'
indx = 5
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)
wtr = 'csv'

pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_test = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
Y_test = np.asarray(pixels_df['pixel_real_value'])
Y_test[0:5]

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test[0:5]

<h2 id="modeling">Modeling (Logistic Regression with Scikit-learn)</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train, Y_train)

Prediction:

In [ ]:
yhat = LR.predict(X_test)
print(len(yhat))
print(len(X_test))

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(Y_test, yhat, average='weighted')

## Making 2d image

In [ ]:
a = np.asarray([1,2,3,4,5,6,7,8,9])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def to_img(lin_arr, r = False):
    _, ax = plt.subplots(figsize = (10,10))
    size = int(round(np.sqrt(len(lin_arr))))
    img = lin_arr.reshape((size, size))
    ax.imshow(img, cmap = 'gray')
    if r:
        return img

Original img

In [ ]:
to_img(Y_test)

Binarized img

In [ ]:
to_img(yhat)

In [ ]:
jaccard_score(Y_test, yhat, average='weighted')

## Gray img

In [ ]:
X_test = np.asarray(pixels_df[['proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
img = to_img(X_test, True)

In [ ]:
def binarize(signal, threshold=0.5):
    signal = np.asarray(signal)
    return signal > threshold

In [ ]:
bin_img = binarize(img)
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bin_img, cmap = 'gray')

In [ ]:
Y_binarized = np.concatenate(bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized, average='weighted')

## Otsu

In [ ]:
def otsu(gray):
    pixel_number = gray.shape[0] * gray.shape[1]
    mean_weigth = 1.0/pixel_number
    his, bins = np.histogram(gray, np.arange(0,257))
    final_thresh = -1
    final_value = -1
    intensity_arr = np.arange(256)
    for t in bins[1:-1]: # This goes from 1 to 254 uint8 range (Pretty sure wont be those values)
        pcb = np.sum(his[:t])
        pcf = np.sum(his[t:])
        Wb = pcb * mean_weigth
        Wf = pcf * mean_weigth

        mub = np.sum(intensity_arr[:t]*his[:t]) / float(pcb)
        muf = np.sum(intensity_arr[t:]*his[t:]) / float(pcf)
        #print mub, muf
        value = Wb * Wf * (mub - muf) ** 2

        if value > final_value:
            final_thresh = t
            final_value = value
    final_img = gray.copy()
    print(final_thresh)
    final_img[gray > final_thresh] = 255
    final_img[gray < final_thresh] = 0
    return final_img

In [ ]:
otsu_bin_img = otsu(img)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(otsu_bin_img, cmap = 'gray')

In [ ]:
Y_binarized_otsu = np.concatenate(otsu_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_otsu, average='weighted')

## Gaus+binarization

In [ ]:
from scipy.ndimage import gaussian_filter as gfilter

In [ ]:
gaus_bin_img = binarize(gfilter(img, 3))

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(gaus_bin_img, cmap = 'gray')

original:

In [ ]:
to_img(Y_test)

In [ ]:
Y_binarized_gaus = np.concatenate(gaus_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_gaus, average='weighted')

In [ ]:
bilateral = cv2.bilateralFilter(np.asarray(img, dtype = np.uint8), 7, 1, 1)
Y_bilaterial = np.concatenate(bilateral)
jaccard_similarity_score(Y_test, Y_bilaterial)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bilateral, cmap = 'gray')

## TEST 2

In [ ]:
tag = 'test2'
indx = 5
dim = 2

pc.count_neighbor_average_array_and_save(dim, indx, tag)
wtr = 'csv'

pixels_df = dm.get_data(dimension=dim, id_indx=indx, what_to_return=wtr, tag=tag)

In [ ]:
pixels_df.head()

In [ ]:
X_test = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
Y_test = np.asarray(pixels_df['pixel_real_value'])
Y_test[0:5]

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test[0:5]

<h2 id="modeling">Modeling (Logistic Regression with Scikit-learn)</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train, Y_train)

Prediction:

In [ ]:
yhat = LR.predict(X_test)
print(len(yhat))
print(len(X_test))

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(Y_test, yhat, average='weighted')

## Making 2d image

In [ ]:
a = np.asarray([1,2,3,4,5,6,7,8,9])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def to_img(lin_arr, r = False):
    _, ax = plt.subplots(figsize = (10,10))
    size = int(round(np.sqrt(len(lin_arr))))
    img = lin_arr.reshape((size, size))
    ax.imshow(img, cmap = 'gray')
    if r:
        return img

Original img

In [ ]:
to_img(Y_test)

Binarized img

In [ ]:
to_img(yhat)

In [ ]:
jaccard_score(Y_test, yhat, average='weighted')

## Gray img

In [ ]:
X_test = np.asarray(pixels_df[['proc_phantom_pixel_values']])
X_test[0:5]

In [ ]:
img = to_img(X_test, True)

In [ ]:
def binarize(signal, threshold=0.5):
    signal = np.asarray(signal)
    return signal > threshold

In [ ]:
bin_img = binarize(img)
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bin_img, cmap = 'gray')

In [ ]:
Y_binarized = np.concatenate(bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized, average='weighted')

## Otsu

In [ ]:
def otsu(gray):
    pixel_number = gray.shape[0] * gray.shape[1]
    mean_weigth = 1.0/pixel_number
    his, bins = np.histogram(gray, np.arange(0,257))
    final_thresh = -1
    final_value = -1
    intensity_arr = np.arange(256)
    for t in bins[1:-1]: # This goes from 1 to 254 uint8 range (Pretty sure wont be those values)
        pcb = np.sum(his[:t])
        pcf = np.sum(his[t:])
        Wb = pcb * mean_weigth
        Wf = pcf * mean_weigth

        mub = np.sum(intensity_arr[:t]*his[:t]) / float(pcb)
        muf = np.sum(intensity_arr[t:]*his[t:]) / float(pcf)
        #print mub, muf
        value = Wb * Wf * (mub - muf) ** 2

        if value > final_value:
            final_thresh = t
            final_value = value
    final_img = gray.copy()
    print(final_thresh)
    final_img[gray > final_thresh] = 255
    final_img[gray < final_thresh] = 0
    return final_img

In [ ]:
otsu_bin_img = otsu(img)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(otsu_bin_img, cmap = 'gray')

In [ ]:
Y_binarized_otsu = np.concatenate(otsu_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_otsu, average='weighted')

## Gaus+binarization

In [ ]:
from scipy.ndimage import gaussian_filter as gfilter

In [ ]:
gaus_bin_img = binarize(gfilter(img, 3))

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(gaus_bin_img, cmap = 'gray')

original:

In [ ]:
to_img(Y_test)

In [ ]:
Y_binarized_gaus = np.concatenate(gaus_bin_img)

In [ ]:
jaccard_score(Y_test, Y_binarized_gaus, average='weighted')

In [ ]:
bilateral = cv2.bilateralFilter(np.asarray(img, dtype = np.uint8), 7, 1, 1)
Y_bilaterial = np.concatenate(bilateral)
jaccard_similarity_score(Y_test, Y_bilaterial)

In [ ]:
_, ax = plt.subplots(figsize = (7,7))
ax.imshow(bilateral, cmap = 'gray')